In [1]:
import numpy as np
import networkx as nx
import time
import math
import pandas as pd
import scipy as sp
import plotly.express as px
import plotly.graph_objs as go
from scipy.sparse import *
from scipy import linalg
from scipy.sparse.linalg import norm
from scipy.optimize import least_squares

## Arnoldi 

This is a copy of the algorithm defined and tested in the notebook `algo2_testing`. It's an implementation of the Algorithm 2 from the paper. It's needed in this notebook since this function is called by the `algo4` function. It's implemented to return exactly what's needed in the `algo4` function.

Everything will be reorganized in the main.py file once everything is working.

In [2]:
def Arnoldi(A,v0,m):
    v = v0
    beta = np.linalg.norm(v)
    v = v/beta
    H = sp.sparse.lil_matrix((m+1,m)) 
    V = sp.sparse.lil_matrix((A.shape[0],m+1))
    V[:,0] = v # each column of V is a vector v

    for j in range(m):
        w = A @ v  
        for i in range(j):
            H[i,j] = v.T @ w # tmp is a 1x1 matrix, so it's O(1) in memory
            w = w - H[i,j]*v 
            
        H[j+1,j] = np.linalg.norm(w)

        if H[j+1,j] == 0:
            # print("Arnoldi breakdown")
            m = j
            v = 0
            break
        else:
            if j < m-1:
                v = w/H[j+1,j]
                V[:,j+1] = v

    return V, H, beta, j  

# Algorithm 4 testing

This algorithm is based on the "Algorithm 4" of the paper, the pseudocode provided by the authors is the following 

![](https://i.imgur.com/H92fru7.png)

In [3]:
def compute_gamma(res, a, k): # function to compute gamma
    gamma = np.ones(len(a))
    for i in range(len(a)):
        if i != k:
            gamma[i] = (res[i]*a[k])/(res[k]*a[i])
        else:
            gamma[i] = 0
    return gamma

Basic test case with random numbers to test the algorithm.

In [66]:
def compute_ptilde(G: nx.Graph):
    """
    Compute the ptilde matrix and the probability vector v
    :param G: the graph
    :return: the ptilde matrix and the probability vector v

    """

    # given the graph G, return it's sparse matrix representation
    A = nx.to_scipy_sparse_array(G, format='lil')

    # create a vector d (sparse), where d[i] = 1 if the i-th row of A not null, else 0
    d = sp.sparse.lil_matrix((1, A.shape[0]))
    for i in range(A.shape[0]):
        # s is the sum of the i-th row of A
        s = sp.sparse.lil_matrix.sum(A[[i],:])
        if s == 0:
            d[0,[i]] = 0

    # probability vector v = 1/n
    v = np.repeat(1/A.shape[0], A.shape[0])

    # initialize the ptilde matrix
    P = sp.sparse.lil_matrix((A.shape[0], A.shape[1]))

    # P(i,j) = 1/(number of non null entries in column j) if A(i,j) != 0, else 0
    for j in range(A.shape[1]):
        for i in range(A.shape[0]):
            if A[i,j] != 0:
                P[i,j] = 1/sp.sparse.lil_matrix.sum(A[:,[j]] != 0)

    Pt = P + v @ d.T

    return Pt, v 

In [85]:
def Algo4(Pt, v, m, a: list, tau, maxit: int, x):

    mv, iter = 0, 1 # mv is the number of matrix-vector products, iter is the number of iterations
    
    # initialize x as a random sparse matrix. Each col is the pagerank vector for a different alpha
    x = sp.sparse.lil_matrix((Pt.shape[0], len(a)))

    # initialize the identity matrix of size Pt.shape
    I = sp.sparse.eye(Pt.shape[0], Pt.shape[1], format='lil')

    # compute the residual vector, it is a matrix of size (n, len(a)). Each col is the residual vector for a different alpha. 
    r = sp.sparse.lil_matrix((Pt.shape[0], len(a)))
    res = np.zeros(len(a))

    # compute the residual vector and the norm of each col in the vector res
    for i in range(len(a)):
        r[:,[i]] = sp.sparse.linalg.spsolve(I - a[i]*Pt, v)
        col = r[:,[i]].toarray()
        res[i] = np.linalg.norm(col)

    # this is a while loop in the paper
    for _ in range(maxit):
        # check if we have converged
        err = np.absolute(np.max(res))
        if err < tau:
            print("Computation ended successfully in ", iter, " iterations and ", mv, " matrix-vector products.")
            return x, iter, mv

        print("\niter = ", iter)
        print("res: ", res)
        print("err = ", err)

        # find k as the index of the largest residual
        k = int(np.argmax(res))
        print("k = ", k)

        # compute gamma as defined in the paper
        gamma = compute_gamma(res, a, k)
       
        # Run Arnoldi
        r_k = r[:,[k]].toarray() # r_k is the residual vector for alpha_k
        A_arnoldi = (1/a[k])*I - Pt # A_arnoldi is the matrix used in Arnoldi
        V, H, beta, j = Arnoldi((1/a[k])*I - Pt, r_k, m) # V is the matrix of vectors v, H is the Hessenberg matrix, beta is the norm of the last vector v, j is the number of iterations of Arnoldi
        H = H[:-1,:] # remove the last row of H
        V = V[:,:-1] # remove the last col of V
        mv = mv + j # update the number of matrix-vector products

        H_e1 = np.zeros(H.shape[0]) 
        H_e1[0] = 1 # canonical vector e1 of size H.shape[0]

        # compute y as the minimizer of || beta*e1 - Hy ||_2 using the least squares method
        y = sp.sparse.lil_matrix((H.shape[1],len(a))) # y is the matrix of vectors y, each col is the vector y for a different alpha

        # we only need the k-th col of y in this iteration
        y[:,[k]] = sp.sparse.linalg.lsqr(H, beta*H_e1)[0]
        y_k = y[:,[k]].toarray()

        # # Update x
        x_new = x
        x_new[:,[k]] = x[:,[k]] + V @ y_k

        V_e1 = np.zeros(V.shape[0])
        V_e1[0] = 1 # canonical vector e1 of size V.shape[0]

        # Update res[k]
        norm_k =np.linalg.norm(beta*V_e1 - V @ y_k) # this returns a scalar
        res[k] = a[k]*norm_k

        # multi shift
        for i in range(len(a)):
            if i != k and res[i] >= tau:
                if res[i] >= tau:
                    
                    H = H + ((1-a[i])/a[i] - (1-a[k])/a[k])*sp.sparse.eye(H.shape[0], H.shape[1], format='lil')

                    # Compute z as described in the paper
                    z1 = H_e1*beta
                    z1 = z1.reshape(z1.shape[0],1)
                    z2 = H @ y[:,[1]]
                    z2 = z2.reshape(z2.shape[0],1)
                    z = z1 - z2

                    # Solve the linear system for A and b
                    A = sp.sparse.hstack([H, z])
                    b = (beta*H_e1)

                    # use the least squares method to solve the linear system
                    to_split = sp.sparse.linalg.lsqr(A, b.reshape(b.shape[0],1))[0]
                    
                    # the last element of to_split is the last element of gamma[i], the other elements are the elements of y[:[i]]
                    y[:,[i]] = to_split[:-1]
                    gamma[i] = to_split[-1]

                    # update x
                    x_new[:,i] = x[:,i] + V @ y[:,[i]]

                    # update the residual vector
                    # print("\tupdating res[", i, "]")
                    # print("\tgamma[", i, "] = ", gamma[i])
                    # print("\tres[", k, "] = ", res[k])
                    # print("\ta[", i, "] = ", a[i])
                    # print("\ta[", k, "] = ", a[k])
                    res[i] = (a[i]/a[k])*gamma[i]*res[k]
                    # print("\tupdated res[", i, "] = ", res[i])
                    # print()

                else:
                    if res[i] < tau:
                        print("res[", i, "] is smaller than tau = ", tau, " at iteration ", iter)

        iter = iter + 1
        x = x_new

    raise Exception('Maximum number of iterations reached')

In [86]:
G = nx.watts_strogatz_graph(1000, 4, 0.1)
Pt, v = compute_ptilde(G)
# a = [0.85, 0.86, 0.87, 0.88, 0.89, 0.90, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99]
a = [0.85, 0.86, 0.87, 0.88]
tau = 1e-6
maxit = 100
n = len(G.nodes)
x = sp.sparse.random(n, len(a), density=0.1, format='lil')

In [87]:
x, iter, mv = Algo4(Pt, v, 100, a, tau, maxit, x)

/usr/lib/python3.10/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:168: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',




iter =  1
res:  [0.21235414 0.22756312 0.24511308 0.26558937]
err =  0.26558937251088227
k =  3


iter =  2
res:  [ 0.81703891  0.8829876   0.10190349 10.69433448]
err =  10.69433447757171
k =  3


iter =  3
res:  [ 0.81703891  0.8829876   0.10190349 10.69433448]
err =  10.69433447757171
k =  3


iter =  4
res:  [ 0.81703891  0.8829876   0.10190349 10.69433448]
err =  10.69433447757171
k =  3


iter =  5
res:  [ 0.81703891  0.8829876   0.10190349 10.69433448]
err =  10.69433447757171
k =  3


iter =  6
res:  [ 0.81703891  0.8829876   0.10190349 10.69433448]
err =  10.69433447757171
k =  3


iter =  7
res:  [ 0.81703891  0.8829876   0.10190349 10.69433448]
err =  10.69433447757171
k =  3


iter =  8
res:  [ 0.81703891  0.8829876   0.10190349 10.69433448]
err =  10.69433447757171
k =  3


iter =  9
res:  [ 0.81703891  0.8829876   0.10190349 10.69433448]
err =  10.69433447757171
k =  3


iter =  10
res:  [ 0.81703891  0.8829876   0.10190349 10.69433448]
err =  10.69433447757171
k =  3



KeyboardInterrupt: 